# Merging Data

Combining and reshaping data from multiple sources

For this module, we will be exploring the techniques for combining and reshaping data from multiple sources. You will learn about:

* Combining data using pandas library
* Validating Merges
* Debugging Chains
* Exporting to Excel


In [ ]:
import numpy as np
import pandas as pd
import zipfile

In [ ]:
# https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data?resource=download
with zipfile.ZipFile('data/AB_NYC_2019.csv.zip') as zip:
  print(zip.namelist())

In [ ]:
ab = pd.read_csv('data/AB_NYC_2019.csv.zip', dtype_backend='pyarrow',
                engine='pyarrow')
ab

In [ ]:
temps = pd.read_csv('data/nyc-ab-temp.csv', index_col=0, dtype_backend='pyarrow',
                engine='pyarrow')

In [ ]:
temps

## Merging

In [ ]:
# Error expected here. We're working to fix it in the next few cells.

(ab
 .merge(temps)
)

In [ ]:
(ab.columns.intersection(temps.columns))

In [ ]:
ab.columns

In [ ]:
temps.columns

In [ ]:
(ab
 .merge(temps, left_on=['latitude', 'longitude'],
        right_on=['lat', 'lon']
       )
)

## Validating Merges

In [ ]:
left = pd.DataFrame({'name': ['Ravi', 'Jose', 'Jose'],
                    'pet': ['Dog', 'Cat', 'Dog']})
right = pd.DataFrame({'name': ['Ravi', 'Jose', 'Sally'],
                      'age': [10, 17, 5]})

In [ ]:
display(left)
display(right)

In [ ]:
left.merge(right, how='inner')

In [ ]:
left.merge(right, how='left')

In [ ]:
left.merge(right, how='right')

In [ ]:
left.merge(right, how='outer')

In [ ]:
left.merge(right, how='cross')

In [ ]:
left

In [ ]:
left.merge(right, how='inner', validate='1:1')

In [ ]:
left.merge(right, how='inner', validate='m:1')

## Debugging Trick

In [ ]:
def limit(df, n_rows, n_cols):
  return df.iloc[:n_rows, :n_cols]

(ab
 .pipe(limit, n_rows=3, n_cols=6)
)

In [ ]:
# add debug to helpers
def debug(df, extra=''):
  print(f'{extra} {df.shape=}')
  return df

(ab
 .pipe(debug, extra='before')
 .merge(temps, left_on=['latitude', 'longitude'],
        right_on=['lat', 'lon']
       )
 .pipe(debug, extra='after') 
)

In [ ]:
(ab
 .pipe(debug, extra='before')
 .merge(temps, left_on=['latitude', 'longitude'],
        right_on=['lat', 'lon']
       )
 .pipe(debug, extra='after') 
 .groupby('neighbourhood_group')
 .mean()
 .pipe(debug, extra='summary') 
)

In [ ]:
# pandas 2 requires numeric_only
(ab
 .pipe(debug, extra='before')
 .merge(temps, left_on=['latitude', 'longitude'],
        right_on=['lat', 'lon']
       )
 .pipe(debug, extra='after') 
 .groupby('neighbourhood_group')
 .mean(numeric_only=True)
 .pipe(debug, extra='summary') 
)

## Cleanup Columns

In [ ]:
(ab
 .pipe(debug, extra='before')
 .merge(temps, left_on=['latitude', 'longitude'],
        right_on=['lat', 'lon']
       )
 .pipe(debug, extra='after') 
 .columns
)

In [ ]:
# Explicitly list columns
(ab
 .pipe(debug, extra='before')
 .merge(temps, left_on=['latitude', 'longitude'],
        right_on=['lat', 'lon']
       )
 .pipe(debug, extra='after') 
 .loc[:, ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'temp']]
 .pipe(debug, extra='limit cols')  
)

## Export to Excel

In [ ]:
xls_out = pd.ExcelWriter('data/airbnb.xlsx')
xl = (ab
 .pipe(debug, extra='before')
 .merge(temps, left_on=['latitude', 'longitude'],
        right_on=['lat', 'lon']
       )
 .pipe(debug, extra='after') 
 .loc[:, ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'temp']]
)

xl.to_excel(xls_out, sheet_name='all')
(xl
 .query('neighbourhood_group=="Brooklyn"')
 .to_excel(xls_out, sheet_name='Brookly')
)
xls_out.close()

In [ ]:
!open data/airbnb.xlsx